In [1]:
using Revise
using PolynomialOptimization, GALAHAD

In [2]:
L = methods(PolynomialOptimization.LANCELOT_simple)[1].module

PolynomialOptimizationLancelot

# Example of use
$$\min f(x_1, x_2, x_3) = x_1^2 + x_2 \sin(x_1 + x_3) + 3x_2^4 x_3^4 + x_2 + 2x_1^2 x_2^2$$
such that
$$c(x_1, x_2, x_3) = \cos(x_1 + 2x_1 - 1) = 0 \\
-1 \leq x_2 \leq 1 \\
1 \leq x_3 \leq 2$$

## Groups
1. $g_1(\alpha) = \alpha^2$ (non-trivial), linear element function $x_1$
2. $g_2(\alpha) = \alpha$ (trivial), nonlinear element function $e_1(x_1, x_2, x_3) = x_2 \sin(x_1 + x_3)$, elemental variables $v_1 = x_2$, $v_2 = x_1$, $v_3 = x_3$, internal variables $u_1 = v_1$, $u_2 = v_2 + v_3$
3. $g_3(\alpha) = \alpha^4$ (non-trivial), weight $3$, nonlinear element function $e_2(x_2, x_3) = x_2 x_3$, elemental variables $v_1 = x_2$, $v_2 = x_3$
4. $g_4(\alpha) = \alpha$ (trivial), linear element function $x_2$
5. $g_5(\alpha) = \alpha^2$ (non-trivial), weight $2$, same type as group 1, nonlinear element function $e_3(x_1, x_2) = x_1 x_2$, same type as $e_2$, elemental variables $v_1 = x_1$, $v_2 = x_2$
6. $g_6(\alpha) = \cos\alpha$ (non-trivial), linear element function $x_1 + 2x_2 - 1$

In [3]:
prob = L.LANCELOT_problem_type(
    # As the first, third, fifth and sixth group functions are non-trivial, we set GXEQX as:
    GXEQX=L.FortranBool[false, true, false, true, false, false],
    # Non-trivial groups 1 and 5 involve functions of the same type (we label this a "type 1"
    # group; the actual label is unimportant, so long as groups of different types have
    # different labels), and these are different from those for groups 2 and 6 (which we call
    # "types 2" and "3")—by convention, trivial groups are of "type 0". So we set ITYPEG as:
    ITYPEG=Cint[1, 0, 2, 0, 1, 3],
    # The first five groups are associated with the objective function while the sixth
    # defines a constraint so we set KNDOFG as:
    KNDOFG=Cint[1, 1, 1, 1, 1, 2],
    # The first nonlinear element occurs in group 2, the second in group 3 and the last in
    # group 5; each element is unweighted. We thus set IELING, ESCALE and ISTADG as:
    IELING=Cint[1, 2, 3],
    ESCALE=Cdouble[1, 1, 1],
    ISTADG=Cint[1, 1, 2, 3, 3, 4, 4],
    # The first nonlinear element function is not a quadratic function of its internal
    # variables so we set control.quadratic_problem=false. The second and third nonlinear
    # element functions are of the same type (we number this a "type 2" element), while the
    # first nonlinear element is different (a "type 1" element). So we set ITYPEE as:
    ITYPEE=Cint[1, 2, 2],
    # Nonlinear element one, e_1 assigns variables x_2, x_1 and x_3 to its elemental
    # variables, while the second and third nonlinear elements e_2 and e_3 assign variables
    # x_2 and x_3, and x_1 and x_2 to their elemental variables, respectively. Hence
    # IELVAR contains:
    IELVAR=Cint[2, 1, 3, 2, 3, 1, 2],
    # In this vector, we now locate the position of the first variable of each nonlinear
    # element, and build the vector ISTAEV as follows:
    ISTAEV=Cint[1, 4, 6, 8],
    # All three nonlinear elements have two internal variables — elements two and three using
    # their elemental variables as internals — so we set INTVAR as follows:
    INTVAR=Cint[2, 2, 2, -1],
    # As the first nonlinear element has a useful transformation between elemental and
    # internal variables, while the other two do not, INTREP contains:
    INTREP=L.FortranBool[true, false, false],
    # Turning to the linear elements, groups one, four and six each have a linear element.
    # Those from groups one and four involve a single variable — x_1 for group one and x_2
    # for group four. That from group six involves the variables x_1 and x_3. All the
    # coefficients, excepting the last, are one; the last is two. We thus set ISTADA as:
    ISTADA=Cint[1, 2, 2, 2, 3, 3, 5],
    # and ICNA and A as
    ICNA=Cint[1, 2, 1, 2],
    A=Cdouble[1, 1, 1, 2],
    # Only the last group has a constant term so B is set to:
    B=Cdouble[0, 0, 0, 0, 0, 1],
    # The bounds for the problem are set in BL and BU. As the first variable is allowed to
    # take any value, we specify lower and upper bounds of ±10^20. Thus we set
    BL=Cdouble[-1e20, -1, -1],
    BU=Cdouble[1e20, 1, 2],
    # The 3rd group is scaled by 3.0, the fifth by 2.0, while the others are unscaled. Thus
    # we set GSCALE to:
    GSCALE=Cdouble[1, 1, 3, 1, 2, 1],
    # It is unclear that the variables are badly scaled so we set VSCALE to:
    VSCALE=Cdouble[1, 1, 1],
    # Finally, we choose to allocate the names Obj 1 to Obj 5 to the four groups associated
    # with the objective function and call the constraint group Constraint. The variables
    # are called x 1 to x 3. We thus set GNAMES and VNAMES to:
    GNAMES=["Obj 1", "Obj 2", "Obj 3", "Obj 4", "Obj 5", "Constraint"],
    VNAMES=["x 1", "x 2", "x 3"],
    # Notice that, in our example, neither the group nor nonlinear element functions depends
    # upon parameters, so both EPVALU and GPVALU are empty, and consequently all components
    # of the parameter starting address arrays ISTEPA and ISTGPA should be set to 0.
    EPVALU=Cdouble[],
    GPVALU=Cdouble[],
    ISTEPA=Cint[0, 0, 0, 1],
    ISTGPA=Cint[0, 0, 0, 0, 0, 0, 1],
    X=Cdouble[0, 0, 1.5],
    Y=Vector{Cdouble}(undef, 6), # Y[end] = 0??
    C=Vector{Cdouble}(undef, 6)
)

PolynomialOptimizationLancelot.LANCELOT_problem_type(3, 6, 3, Int32[1, 2, 3], Int32[1, 1, 2, 3, 3, 4, 4], Int32[2, 1, 3, 2, 3, 1, 2], Int32[1, 4, 6, 8], Int32[2, 2, 2, -1], Int32[-1183694016, 627, 1650756768, 32760], Int32[1, 2, 1, 2], Int32[1, 2, 2, 2, 3, 3, 5], Int32[1, 1, 1, 1, 1, 2], Int32[1, 2, 2], Int32[0, 0, 0, 1], Int32[1, 0, 2, 0, 1, 3], Int32[0, 0, 0, 0, 0, 0, 1], [1.0, 1.0, 1.0, 2.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [-1.0e20, -1.0, -1.0], [1.0e20, 1.0, 2.0], [0.0, 0.0, 1.5], [5.0e-324, 5.0e-324, 5.0e-324, 5.0e-324, 5.0e-324, 5.0e-324], [6.95177592075344e-310, 6.951775920691e-310, 6.9517759206554e-310, 6.9517398802622e-310, 6.95174279498705e-310, 6.95174279456097e-310], [1.0, 1.0, 3.0, 1.0, 2.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0], Float64[], Float64[], PolynomialOptimizationLancelot.FortranBool[false, true, false, true, false, false], PolynomialOptimizationLancelot.FortranBool[true, false, false], PolynomialOptimizationLancelot.FortranString{10}["x 1       ", "x 2       "

Notice that, in our example, neither the group nor nonlinear element functions depends upon parameters, so both `prob.EPVALU` and `prob.GPVALU` are empty, and consequently all components of the parameter starting address arrays `prob.ISTEPA` and `prob.ISTGPA` should be set to `0`.
LANCELOT solve leaves the computation of function and derivative values of the group and nonlinear element functions to the user—this may either be done internally (by providing suitable `GROUP` and `ELFUN` callbacks), or externally via reverse communication. We need only to specify the group functions for the non-trivial groups, groups 1, 3 and 5. Also note that the function and gradient values are only evaluated if the component is specified in `ICALCG`.

A suitable GROUP subroutine might be the following:

In [4]:
function GROUP(GVALUE, FVALUE, GPVALU, ncalcg, ITYPEG, ISTGPA, ICALCG, derivs)
    @inbounds for jcalcg in 1:ncalcg
        igroup = ICALCG[jcalcg]
        igrtyp = ITYPEG[igroup]
        iszero(igrtyp) && continue # skip if the group is trivial
        ipstrt = ISTGPA[igroup] -1
        if igrtyp == 1
            α = FVALUE[igroup]
            if !derivs
                GVALUE[igroup, 1] = α * α
            else
                GVALUE[igroup, 2] = 2α
                GVALUE[igroup, 3] = 2
            end
        elseif igrtyp == 2
            α = FVALUE[igroup]
            α² = α * α
            if !derivs
                GVALUE[igroup, 1] = α² * α²
            else
                GVALUE[igroup, 2] = 4α² * α
                GVALUE[igroup, 3] = 12α²
            end
        elseif igrtyp == 3
            α = FVALUE[igroup]
            if !derivs
                GVALUE[igroup, 1] = cos(α)
            else
                GVALUE[igroup, 2] = -sin(α)
                GVALUE[igroup, 3] = -cos(α)
            end
        end
    end
    return 0
end;

GROUP (generic function with 1 method)

Here, the boolean parameter `derivs` set `true` specifies that the first and second derivatives are required; a `false` value will return the function values.

To evaluate the values and derivatives of the nonlinear element functions, the following `ELFUN` subroutine would be suitable:

In [5]:
function ELFUN(FUVALS, XVALUE, EPVALU, ncalcf, ITYPEE, ISTAEV, IELVAR, INTVAR, ISTADH, ISTEPA, ICALCF, ifflag)
    @inbounds for jcalcf in 1:ncalcf
        ielemn = ICALCF[jcalcf]
        ilstrt = ISTAEV[ielemn] -1
        igstrt = INTVAR[ielemn] -1
        ipstrt = ISTEPA[ielemn] -1
        if ifflag == 3
            ihstrt = ISTADH[ielemn] -1
        end
        ieltyp = ITYPEE[ielemn]
        if ieltyp == 1
            u1 = XVALUE[IELVAR[ilstrt+1]]
            u2 = XVALUE[IELVAR[ilstrt+2]]
            u3 = XVALUE[IELVAR[ilstrt+3]]
            v1 = u1
            v2 = u2 + u3
            cs = cos(v2)
            sn = sin(v2)
            if ifflag == 1
                FUVALS[ielemn] = v1 * sn
            else
                FUVALS[igstrt+1] = sn
                FUVALS[igstrt+2] = v1 * cs
                if ifflag == 3
                    FUVALS[ihstrt+1] = 0
                    FUVALS[ihstrt+2] = cs
                    FUVALS[ihstrt+3] = -v1 * sn
                end
            end
        elseif ieltyp == 2
            u1 = XVALUE[IELVAR[ilstrt+1]]
            u2 = XVALUE[IELVAR[ilstrt+2]]
            if ifflag == 1
                FUVALS[ielemn] = u1 * u2
            else
                FUVALS[igstrt+1] = u2
                FUVALS[igstrt+2] = u1
                if ifflag == 3
                    FUVALS[ihstrt+1] = 0
                    FUVALS[ihstrt+2] = 1
                    FUVALS[ihstrt+3] = 0
                end
            end
        end
    end
    return 0
end;

ELFUN (generic function with 1 method)

The integer parameter `ifflag` specifies whether function values (`ifflag = 1`) or derivatives (`ifflag > 1`) are required. When `ifflag = 2`, just first derivatives are required, while `ifflag = 3` also requests second derivatives. Notice that the derivatives are taken with respect to the internal variables. For the first nonlinear element function, this means that we must transform from elemental to internal variables before evaluating the derivatives. Thus, as this function may be written as $f(\nu_1, \nu_2) = \nu_1 \sin\nu_2$, where $\nu_1 = u_1$ and $\nu_2 = u_2 + u_3$, the gradient and Hessian matrix are
$$\begin{pmatrix} \sin\nu_2 \\ \nu_1\cos\nu_2 \end{pmatrix}
\quad\text{and}\quad
\begin{pmatrix} 0 & \cos\nu_2 \\ \cos\nu_2 & -\nu_1\sin\nu_2 \end{pmatrix}\text,$$
respectively. Notice that it is easy to specify the second derivatives for this example so we do so. Also note that the function and gradient values are only evaluated if the component is specified in `ICALCF`. We must also specify the routine `RANGE` for our example. As we have observed, only the first element has a useful transformation. The transformation matrix is
$$\begin{pmatrix} 1 & 0 & 0 \\ 0 & 1 & 1 \end{pmatrix}\text.$$
As a consequence, the following routine `RANGE` is appropriate:

In [6]:
function RANGE(ielemn, transp, W1, W2, nelvar, ninvar, ieltyp)
    @inbounds if ieltyp == 1 # Element type 1 has a non-trivial transformation
        if transp
            W2[1] = W1[1]
            W2[2] = W1[2]
            W2[3] = W1[2]
        else
            W2[1] = W1[1]
            W2[2] = W1[2] + W1[3]
        end
    end
    # Element 2 has a trivial transformation - no action required
end;

RANGE (generic function with 1 method)

This completes the supply of information to LANCELOT solve. The problem may now be solved using the following program. We choose to solve the model subproblem using a preconditioned conjugate gradient scheme, using diagonal preconditioning, a "box" shaped trust-region, calculating an exact Cauchy point at each iteration but not obtaining an accurate minimizer of the model within the box. Furthermore, we start from the initial estimate $x_1 = 0.0$, $x_2 = 0.0$ and $x_3 = 1.5$. As we have no idea of a good estimate for the Lagrange multiplier associated with the constraint (group 6), we set $y_6 = 0$. We observe that the objective function is bounded below by $-2.0$ within the feasible box and terminate when the norms of the reduced gradient of the Lagrangian function and constraints are smaller than $10^{-5}$.

In [7]:
data, control = L.LANCELOT_initialize()
info = L.LANCELOT_inform_type()

PolynomialOptimizationLancelot.LANCELOT_inform_type(0, 0, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 1.7976931348623157e308, 1.7976931348623157e308, 1.7976931348623157e308, 0.0, 0.0, 0.0, 0.0, 0.0, false, (32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32), PolynomialOptimizationLancelot.SCU_inform_type(0, 0, (0, 0, 0)), PolynomialOptimizationLancelot.SILS_ainfo(0, 0, 0, -1, -1, -1, -1, -1, -1, 0, 0, 0, -1, 0, 0, -1.0, -1.0), PolynomialOptimizationLancelot.SILS_finfo(0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1.0, -1.0, -1.0, -1.0, 0.0, 0.0), PolynomialOptimizationLancelot.SILS_sinfo(-1, -1, -1.0, -1.0, -1.0, -1.0, -1.0))

In [8]:
control.maxit = 100
control.out = 6
control.print_level = 1
control.stopg = 0.00001
control.stopc = 0.00001
control.linear_solver = 1
control.exact_gcp = false

info.status = 0;

In [9]:
L.LANCELOT_solve(prob; RANGE, control, inform=info, data, ELFUN, GROUP)

In [53]:
if iszero(info.status)
    println(info.iter, " iterations. Optimal objective value = ", info.obj, ", Optimal solution = ", prob.X)
else
    println("LANCELOT_solve exit status ", info.status)
end

-1 iterations. Optimal objective value = 1.7976931348623157e308, Optimal solution = [0.0, 0.0, 1.5]


In [54]:
prob

PolynomialOptimizationLancelot.LANCELOT_problem_type(3, 6, 3, Int32[1, 2, 3], Int32[1, 1, 2, 3, 3, 4, 4], Int32[2, 1, 3, 2, 3, 1, 2], Int32[1, 4, 6, 8], Int32[2, 2, 2, -1], Int32[#undef], Int32[1, 2, 1, 2], Int32[1, 2, 2, 2, 3, 3, 5], Int32[1, 1, 1, 1, 1, 2], Int32[1, 2, 2], Int32[0, 0, 0, 1], Int32[1, 0, 2, 0, 1, 3], Int32[0, 0, 0, 0, 0, 0, 1], [1.0, 1.0, 1.0, 2.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [-1.0e20, -1.0, -1.0], [1.0e20, 1.0, 2.0], [0.0, 0.0, 1.5], [6.95173925665727e-310, 6.95173925665727e-310, 6.95173925665727e-310, 6.95173925665727e-310, 6.95173925665727e-310, 6.95173925665727e-310], [1.330926457704e-311, 6.95174279455464e-310, 1.3309264577277e-311, 1.3309264577593e-311, 6.95174279455464e-310, 6.95174279455464e-310], [1.0, 1.0, 3.0, 1.0, 2.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0, 1.0], Float64[], Float64[], Int32[0, 1, 0, 1, 0, 0], Int32[1, 0, 0], PolynomialOptimizationLancelot.FortranString{10}["x 1       ", "x 2       ", "x 3       "], PolynomialOptimizationLancelot.FortranStr